### Bi Encoder, Cross Encoder 불러오기


In [2]:
from transformers import ElectraTokenizerFast, ElectraModel, ElectraForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from model import SentenceBert
import pandas as pd
import numpy as np
import utils
import torch

# Bi_Encoder
model = ElectraModel.from_pretrained("../model/bi_encoder_finish")
bi_encoder = SentenceBert(model)

# Cross_Encoder
cross_encoder = ElectraForSequenceClassification.from_pretrained("../model/cross_encoder_STS")

# Tokenizer
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")


### Data 불러오기

In [3]:
# Load Data
raw_data = pd.read_csv("../data/bookList/raw_book_info_list.csv", index_col=0)
englist = pd.read_csv("../data/preprocess/englist.csv")


# Bi eoncoder 전용 Tokenizer
def tokenizing_function_for_bi_encoder(text, max_length=128):
    token = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=max_length,
        stride=20,
        return_overflowing_tokens=True,
        return_tensors="pt",
    )
    token.pop("overflow_to_sample_mapping")
    return token


raw_data.head(5)


,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"['', '텍스트 다루기', '문자열을 줄여 표시하려면 textwrap shorte...",['이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받...,['실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선...
2,도메인 주도 설계 첫걸음,"['장 비즈니스 도메인 분석하기', '비즈니스 도메인이란', '하위 도메인이란', ...","['소프트웨어 개발은 예전보다 어렵다', '개발자로서 지속적으로 변화하는 기술 트렌...",[]
3,그림과 실습으로 배우는 도커 & 쿠버네티스,"['도커란 무엇인가', '안개 속에 숨겨진 도커의 정체는', '데이터나 프로그램을 ...",['이 책은 컨테이너 기술이 어렵게 느껴지는 엔지니어나 백엔드 기술에 자신이 없는 ...,[]
4,눈 떠보니 메타버스 마스터,"['메타버스란', '메타버스의 활용 사례', '순천향대학교 입학식 이프랜드', 'S...","['메타버스 플랫폼', '이프랜드 제페토 게더타운 활용 가이드', '메타버스의 사례...",['불과 얼마 전만 해도 우리는 SF 영화를 넋 놓고 보면서 이게 영화가 아니라 현...


### `그림과 실습으로 배우는 도커 & 쿠버네티스` 도서와 연관있는 도서 추출

 Bi Encoder로 `그림과 실습으로 배우는 도커 & 쿠버네티스` 도서 Embedding 추출



In [4]:
i = 3
query = utils.merge_series_to_str(raw_data.iloc[i], print_on=True)
query = utils.trans_eng_to_han(query, englist=englist)
query = " ".join(query)

# Embedding 추출
token_1 = tokenizing_function_for_bi_encoder(query)
sen_embedding = bi_encoder(**token_1)["sentence_embedding"].detach()
query_embedding = torch.sum(sen_embedding, dim=0) / sen_embedding.size(0)


변환한 도서정보 :  그림과 실습으로 배우는 도커 & 쿠버네티스


도서 전체 Embedding 불러오기


In [5]:
lst = []
for i in range(1, 6):
    x = torch.load(f"save/bi_encoder_doc_{i}.pt", map_location=torch.device("cpu"))
    x = pd.DataFrame(x)
    lst.append(x)


y = pd.concat(lst, axis=0).reset_index(drop=True)


y.head(5)

# rep_embedding : 도서 정보를 담은 embedding


,title,rep_embedding
0,한 권으로 끝내는 메타버스 크리에이터,"[[tensor(0.6947), tensor(0.1192), tensor(-0.46..."
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"[[tensor(0.5697), tensor(-0.0356), tensor(-0.5..."
2,도메인 주도 설계 첫걸음,"[[tensor(0.7111), tensor(0.0659), tensor(-0.48..."
3,그림과 실습으로 배우는 도커 & 쿠버네티스,"[[tensor(0.6220), tensor(-0.0605), tensor(-0.4..."
4,눈 떠보니 메타버스 마스터,"[[tensor(0.6649), tensor(0.1309), tensor(-0.53..."


`그림과 실습으로 배우는 도커 & 쿠버네티스` Embedding과 도서 전체 Embedding 비교하여 후보군 선정

In [6]:
data = y["rep_embedding"].values.tolist()
data = torch.cat(data)

cos_sim = cosine_similarity(query_embedding.unsqueeze(0), data)

idx = np.argsort(cos_sim)[0][::-1]
raw_data.iloc[idx]["book_title"][1:11]


1009                    따라하며 배우는 도커와 CI 환경
3287                         리눅스 서버 관리 바이블
1368               타입스크립트, AWS 서버리스로 들어올리다
419                초보를 위한 젠킨스 2 활용 가이드 2/e
3274                          오픈스택을 다루는 기술
4713         Windows Server Container 시작하기
3580                         도커, 컨테이너 빌드업!
4337    Windows Server 2016 Hyper-V 쿡북 2/e
3535                  빠르게 훑어보는 구글 클라우드 플랫폼
3358           파이썬을 이용한 머신러닝, 딥러닝 실전 개발 입문
Name: book_title, dtype: object

### Cross Encoder로 후보군 ReRanking하기


In [7]:
from tqdm.notebook import tqdm


def rerank_candidates(
    query: str, candidates: pd.DataFrame, top_k=10, max_length=256, sentence_n=4
) -> list:
    """
    query : 찾고자 하는 도서 정보
    candidates : query와 유사한 도서 정보
    top_k : 상위 k개 정보만 추출
    max_length : 도서 비교 시 활용될 문장 길이
    sentence_n : 도서 비교 시 활용될 문장 개수
    """

    lst = []
    for i in tqdm(range(1, len(candidates))):

        # 후보군 전처리
        candidate = utils.merge_series_to_str(candidates.iloc[i])
        candidate = utils.trans_eng_to_han(candidate, englist=englist)
        candidate = " ".join(candidate)

        # Tokenzing
        token = tokenizer(
            query,
            candidate,
            return_tensors="pt",
            max_length=max_length,
            truncation=True,
            padding=True,
            stride=20,
            return_overflowing_tokens=True,
        )
        token.pop("overflow_to_sample_mapping")

        # 도서 비교에 활용 될 문장 샘플링
        torch.manual_seed(42)

        tokens = token.input_ids
        random_int = torch.randint(0, tokens.size(0), size=(sentence_n,))
        tokens = tokens[random_int]

        # Cross Encoder로 문장 유사도 추출
        logit = cross_encoder(tokens)["logits"].detach()
        lst.append((round(torch.mean(logit).item(), 5), candidates.iloc[i]["book_title"]))

        # 연관성 높은 순으로 재졍렬
    lst = sorted(lst, key=lambda x: x[0])[::-1]

    return lst[:top_k]


# Bi_encoder로 확보한 도서 정보 추출
candidates_inform = raw_data.iloc[idx][:20]
candidates = candidates_inform.drop(columns=["book_toc"])

rerank_candidates(query, candidates, top_k=5)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/19 [00:00<?, ?it/s]

[(0.76013, '초보를 위한 젠킨스 2 활용 가이드 2/e'),
 (0.75422, '도커, 컨테이너 빌드업!'),
 (0.73951, '따라하며 배우는 도커와 CI 환경'),
 (0.73613, '헬름 배우기'),
 (0.72709, '서비스 운영이 쉬워지는 AWS 인프라 구축 가이드')]